In [1]:
from harmony_tokenizers_m21 import ChordSymbolTokenizer, PitchClassTokenizer, MelodyPitchTokenizer, MergedMelHarmTokenizer
from data_utils import StructGPTMelHarmDataset, GenCollator
from torch.utils.data import DataLoader
import torch
from transformers import AutoConfig, GPT2LMHeadModel
import numpy as np

from a_star import AStarGPT

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
test_dir = '/media/maindisk/maximos/data/hooktheory_test'

cstok = ChordSymbolTokenizer()
pctok = PitchClassTokenizer()
meltok = MelodyPitchTokenizer()
# tokenizer = MergedMelHarmTokenizer(meltok, cstok)
tokenizer = MergedMelHarmTokenizer(meltok, pctok)

test_dataset = StructGPTMelHarmDataset(test_dir, tokenizer, max_length=512, num_bars=16, return_harmonization_labels=True)

collator = GenCollator(tokenizer)

trainloader = DataLoader(test_dataset, batch_size=1, shuffle=True, collate_fn=collator)

In [3]:
config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer.vocab),
    n_positions=512,
    n_layer=8,
    n_head=8,
    pad_token_id=tokenizer.vocab[tokenizer.pad_token],
    bos_token_id=tokenizer.vocab[tokenizer.bos_token],
    eos_token_id=tokenizer.vocab[tokenizer.eos_token],
    n_embd=512
)

model = GPT2LMHeadModel(config)

# model_path = 'saved_models/gpt/ChordSymbolTokenizer/ChordSymbolTokenizer.pt'
model_path = 'saved_models/gpt/PitchClassTokenizer/PitchClassTokenizer.pt'

# device_name = 'cuda:0'
device_name = 'cpu'
if device_name == 'cpu':
    device = torch.device('cpu')
else:
    if torch.cuda.is_available():
        device = torch.device(device_name)
    else:
        print('Selected device not available: ' + device_name)

checkpoint = torch.load(model_path, map_location=device_name, weights_only=True)
model.load_state_dict(checkpoint)
model.eval()
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(211, 512)
    (wpe): Embedding(512, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-7): 8 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=1536, nx=512)
          (c_proj): Conv1D(nf=512, nx=512)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=2048, nx=512)
          (c_proj): Conv1D(nf=512, nx=2048)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=512, out_features=211, bias=False)
)

In [4]:
batch = next(iter(trainloader))

/home/maximos/miniconda3/envs/torch/lib/python3.12/site-packages/music21/stream/base.py:3694: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


In [ ]:
all_ids = batch['input_ids']
print(all_ids)
melody_end_index = all_ids[0].tolist().index( tokenizer.vocab['</m>'] )
harmony_start_index = all_ids[0].tolist().index( tokenizer.vocab['<h>'] )
print(melody_end_index)
# constraint_ids = all_ids[0][melody_end_index:]
print(constraint_ids)
# start_harmony_position = np.where( all_ids == harmony_start_index )[0][0]
input_ids = all_ids[0][:(harmony_start_index+2)]
constraint_ids = input_ids[melody_end_index:(harmony_start_index+1)]
input_ids = input_ids.reshape(1, -1)
print(input_ids.shape)

tensor([[  2,   6, 183,  98,  70, 106,  68, 114,  66, 122,  66, 126,  68,   6,
          98,  65, 106,  63, 114,  61, 122,  61, 126,  63,   6,  98,  58, 106,
          56, 114,  54, 122,  54, 126,  56,   6,  98,  56, 106,   4, 114,  58,
         118,  61, 122,  63, 126,  66,   6,  98,  70, 106,  68, 114,  66, 122,
          66, 126,  68,   6,  98,  65, 106,  63, 114,  61, 122,  61, 126,  63,
           6,  98,  58, 106,  56, 114,  54, 122,  54, 126,  56,   6,  98,  56,
         106,  54, 110,  56, 122,  54, 126,  56,   6,  98,   4, 106,  54, 110,
          56, 122,  54, 126,  56,   6,  98,  56, 106,  54, 110,  56, 122,  54,
         126,  56,   6,  98,  56, 106,  54, 110,  56, 122,   4,   6,  98,   4,
         114,  70,   8,   6,   9,   6,   9,   6,   9,   6,  98, 201, 204, 208,
           9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,
           9,   6,   9,   3,   7,   6,  98, 201, 204, 208, 209, 126, 201, 204,
         208,   6,  98, 201, 204, 208, 102, 199, 204

In [6]:
astar = AStarGPT( model, tokenizer, input_ids, constraint_ids, max_length=512, beam_width=20, lookahead_k=10 )

In [7]:
print(astar.constraint_bar)
print(astar.position_token)
print(astar.chord_tokens)

4
position_0x00
['chord_pc_2', 'chord_pc_5', 'chord_pc_9']


In [8]:
generated_ids, finished = astar.decode()
generated_tokens = []

children
constraint_checker: ['<h>', '<bar>', 'position_0x00']
num_bars: 1 - num_tokens: 3
new_logprob: -0.010170303285121918
constraint_checker: ['<h>', '<bar>', '<bar>']
num_bars: 2 - num_tokens: 3
new_logprob: -5.352053165435791
constraint_checker: ['<h>', '<bar>', 'position_1x00']
num_bars: 1 - num_tokens: 3
new_logprob: -6.023471355438232
constraint_checker: ['<h>', '<bar>', 'position_2x00']
num_bars: 1 - num_tokens: 3
new_logprob: -6.6419677734375
constraint_checker: ['<h>', '<bar>', 'position_0x50']
num_bars: 1 - num_tokens: 3
new_logprob: -6.965638160705566
constraint_checker: ['<h>', '<bar>', 'position_1x50']
num_bars: 1 - num_tokens: 3
new_logprob: -8.709787368774414
constraint_checker: ['<h>', '<bar>', 'position_3x50']
num_bars: 1 - num_tokens: 3
new_logprob: -8.830560684204102
constraint_checker: ['<h>', '<bar>', 'position_0x25']
num_bars: 1 - num_tokens: 3
new_logprob: -9.490585327148438
constraint_checker: ['<h>', '<bar>', 'position_3x00']
num_bars: 1 - num_tokens: 3
new_

In [9]:
print(generated_ids.shape)

torch.Size([1, 220])


In [10]:
print(len(finished))
print(finished[0].tokens)

1
tensor([[  2,   6, 183,  98,  70, 106,  68, 114,  66, 122,  66, 126,  68,   6,
          98,  65, 106,  63, 114,  61, 122,  61, 126,  63,   6,  98,  58, 106,
          56, 114,  54, 122,  54, 126,  56,   6,  98,  56, 106,   4, 114,  58,
         118,  61, 122,  63, 126,  66,   6,  98,  70, 106,  68, 114,  66, 122,
          66, 126,  68,   6,  98,  65, 106,  63, 114,  61, 122,  61, 126,  63,
           6,  98,  58, 106,  56, 114,  54, 122,  54, 126,  56,   6,  98,  56,
         106,  54, 110,  56, 122,  54, 126,  56,   6,  98,   4, 106,  54, 110,
          56, 122,  54, 126,  56,   6,  98,  56, 106,  54, 110,  56, 122,  54,
         126,  56,   6,  98,  56, 106,  54, 110,  56, 122,   4,   6,  98,   4,
         114,  70,   8,   6,   9,   6,   9,   6,   9,   6,  98, 201, 204, 208,
           9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,   9,   6,
           9,   6,   9,   3,   7,   6,  98, 201, 204, 208,   6,  98, 199, 203,
         206,   6,  98, 201, 204, 209,   6,  98, 2

In [11]:
for i in generated_ids[0]:
    generated_tokens.append( tokenizer.ids_to_tokens[ int(i) ].replace(' ','x') )
print(generated_tokens[harmony_start_index:])

['<h>', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9', '<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9', '<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9', '<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9', '<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7', '<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10', '</s>']


In [12]:
t = generated_tokens[harmony_start_index:]
line = []
for i in range(len(t)):
    line.append(t[i])
    if i+1 < len(t) and 'bar' in t[i+1]:
        print(line)
        line = []
print(line)

['<h>']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9']
['<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9']
['<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9']
['<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_9']
['<bar>', 'position_0x00', 'chord_pc_0', 'chord_pc_4', 'chord_pc_7']
['<bar>', 'position_0x00', 'chord_pc_2', 'chord_pc_5', 'chord_pc_10', '</s>']
